# Decoding with Belief Propagation

We want to iterate the following the following equations:

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu } \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$  

which are Eqs.(96) from the paper [Low-density parity-check codes—A statistical physics perspective](https://www.sciencedirect.com/science/article/pii/S1076567002800180), by R. Vicente, D. Saad and Y. Kabashima.

The function $ \beta(x) $ is the Nishimori temperature,

$$ \beta(x) = \frac{1}{2} \log \left(  \frac{1- \rho}{\rho}  \right) \; \; .  $$

The quantity $\rho$ is the flip probablility of the noisy channel (BSC),

$$ P ( J | J^{(0)} ) = (1 - \rho) \delta_{J, J^{(0)} } + \rho \delta_{J, -J^{(0)} }   \; \; , $$

whereas the prior distribution for each meassage bit is assumed to be

$$ P ( S_j ) = (1 - \rho_\xi) \delta_{+1, S_j }  +  \rho_\xi \delta_{-1, S_j }  \; \; .  $$

The object $ {\cal L} (\mu)$ represents the set of $K$ non-zero elements on the row $\mu$ of the code generator matrix ${\cal G}$ (the one which adds redundancy), 

$$  {\cal L} (\mu) = \langle i_1, i_2, ..., i_K \rangle  \; \; .  $$

The are $C$ non-zero elements per column on the matrix ${\cal G}$:

$$ \sum_{\mu : j \in {\cal L}(\mu)} i_j = C \; \; ; \; \; \forall j = 1, ..., K \; \; . $$

The object $ {\cal M} (j)$ represents the set of all index sets that contain $j$.

After convergence of the iterative procedure, we can calculate the pseudo-posterior:

$$ m_{j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) } \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$
from which the Bayes optimal estimate is obtained
$$ \hat{\xi}_j  = sign( m_j ) $$ 

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

Defining the variables of the problem:

In [2]:
# Message lengh
N = 100

# Codeword lengh
M = 200

# Non-zero elements per row of the generation matrix
K = 4

# Number of messages
n = 10

# Noisy channel
p = 0.3
beta = 0.5*np.log( (1 - p) / p)

# Message prior
p_prior = 0.1
beta_prior = 0.5*np.log( (1 - p_prior) / p_prior)

## Generating messages

Each message is a $N$ dimensional vector. Generate a set of $n$ messages.

In [3]:
random = torch.rand([n, N])
    
message = torch.zeros([n, N])

for j in range(random.shape[0]):
    
    for k in range(random.shape[1]):
               
            #### -1 with probability p_prior
            if random[j,k] <= p_prior:
                message[j,k] = -1.
            #### +1 with probability 1 - p_prior
            else:
                message[j,k] = +1.

In [4]:
message.shape

torch.Size([10, 100])

Each message is encoded to a high dimensional vector ${\bf J}^{(0)} \in \{ \pm 1  \}^M$ defined as 

$$   J^{(0)}_{\langle i_1, i_2, ...., i_K \rangle} = \xi_1 \xi_ 2 ... \xi_K  \; \; ,$$

where $M$ sets of $K \in [ 1, ..., N]$ indexes are randomly chosen.

In [112]:
encoding = torch.randint(0, N, [M, K])

In [113]:
encoding.shape

torch.Size([200, 4])

From `encoding`, we construct the encoded message ${\bf J}^{(0)}$.

In [7]:
for j in range(message.shape[0]):
    
    J0 = torch.take(message[j], encoding).prod(dim=1)

In [8]:
# Initializing
J0 = torch.take(message[0], encoding).prod(dim=1)
J0 = J0.unsqueeze(0)

for j in range(1, message.shape[0]):
    
    J0_ = torch.take(message[j], encoding).prod(dim=1)
    J0_ = J0_.unsqueeze(0)
    
    J0 = torch.cat((J0, J0_), dim= 0)

Now the corrupted version.

In [9]:
J = J0.clone()

In [10]:
random = torch.rand(J.shape)
                      
for j in range(J.shape[0]):
    for k in range(J.shape[1]):
          
        if random[j, k] <= p:
            J[j, k] = -J[j, k]

Let us focus in one received message to iterate the belief propagation equations.

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu} \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right) + \beta(\rho_\xi)    \right)   \; \; ,  $$  

with $j = 1, ..., N$ and $\mu = 1, ..., M$.

We cal this message `J_`. We will worry later about a loop over all the received messages.

In [11]:
J_ = J[0]
print(J_)
print(J_.shape)

tensor([ 1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1.,
         1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,
        -1., -1., -1., -1., -1.,  1.,  1.,  1., -1.,  1., -1., -1., -1.,  1.,
         1.,  1., -1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1., -1.,
         1.,  1., -1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1., -1.,  1., -1., -1., -1., -1., -1., -1., -1.,  1.,
         1.,  1., -1.,  1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1., -1.,
         1.,  1., -1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,
         1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1.,  1., -1., -1.,
         1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1.,  1., -1.,
         1., -1.,  1., -1.,  1., -1., -1.,  1.,  1., -1.,  1., -1.,  1.,  1.,
        -1., -1., -1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1., -1.,  1., -1., -1., -1., -1.,  1., -1., -1., -1., -

Random initialization of the beliefs $m_{\mu l}$.

In [153]:
m = torch.rand(M, N)

In [154]:
m

tensor([[0.5737, 0.3259, 0.1629,  ..., 0.5391, 0.5564, 0.4237],
        [0.8206, 0.2177, 0.8888,  ..., 0.0783, 0.1224, 0.3776],
        [0.6112, 0.9259, 0.9299,  ..., 0.8411, 0.8729, 0.6649],
        ...,
        [0.5682, 0.4077, 0.6779,  ..., 0.6039, 0.6576, 0.7261],
        [0.2614, 0.4347, 0.6711,  ..., 0.1953, 0.9119, 0.3932],
        [0.4544, 0.2197, 0.0443,  ..., 0.8243, 0.3393, 0.3847]])

Initialize an empty tensor to represent $\hat{m}_{\mu l}$.

In [155]:
m_hat = torch.empty(M, N)

In [156]:
m_hat

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

We want to calculate $\hat{m}_{\mu j}$.

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

This first implementation has two `for` loops. This is potentially harmful if one cares about efficienty. We obviously do, but since we are just beginning, lets go on like this.

In [157]:
for mu in range(M):
    for j in range(N):
          
        # Keep only L(mu) which a are different of j
        index_no_j = torch.nonzero(encoding[mu] != j).squeeze()
        L_no_j = encoding[mu][index_no_j]
        
        # Message update        
        m_hat[mu, j] = torch.tanh( beta* J_[mu])*torch.take(m[mu], L_no_j).prod(dim=0)

In [158]:
m_hat

tensor([[ 5.2171e-06,  5.2171e-06,  5.2171e-06,  ...,  5.2171e-06,
          5.2171e-06,  5.2171e-06],
        [ 2.0069e-02,  2.0069e-02,  2.0069e-02,  ...,  2.0069e-02,
          2.0069e-02,  2.0069e-02],
        [-2.1374e-02, -2.1374e-02, -2.1374e-02,  ..., -2.1374e-02,
         -2.1374e-02, -2.1374e-02],
        ...,
        [ 1.1966e-03,  1.1966e-03,  1.1966e-03,  ...,  1.1966e-03,
          1.1966e-03,  1.1966e-03],
        [ 2.8509e-03,  2.8509e-03,  2.8509e-03,  ...,  2.8509e-03,
          2.8509e-03,  2.8509e-03],
        [ 1.1827e-02,  1.1827e-02,  1.1827e-02,  ...,  1.4347e-02,
          1.1827e-02,  1.1827e-02]])

The next step is to implement:

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu} \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right) + \beta(\rho_\xi)     \right)  \; \; ,  $$ 

In [159]:
for j in range(N):
    
    for mu in range(M):
        
        M_set = torch.where(encoding == j)[0]
        
        M_set_no_mu = M_set[torch.nonzero(M_set != mu).squeeze()]
        
        m[mu, j] = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set_no_mu).sum() + beta_prior)

In [162]:
m

tensor([[0.8080, 0.6590, 0.7828,  ..., 0.9041, 0.6813, 0.7584],
        [0.8080, 0.6590, 0.7828,  ..., 0.9041, 0.6813, 0.7584],
        [0.8080, 0.6590, 0.7828,  ..., 0.9041, 0.6813, 0.7584],
        ...,
        [0.8080, 0.6590, 0.7828,  ..., 0.9041, 0.6813, 0.7584],
        [0.8080, 0.6590, 0.7828,  ..., 0.9041, 0.6813, 0.7584],
        [0.8080, 0.6590, 0.7828,  ..., 0.9014, 0.6813, 0.7584]])

The next step is to implement the pseudo-posterior, which is calculated after convergence of the messages above:

$$ m_{j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) } \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$

In [170]:
m_bayes = []

for j in range(N):
    
    M_set = torch.where(encoding == j)[0]
       
    m_j = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set).sum() + beta_prior)
    
    m_bayes.append(m_j.item())
            
m_bayes

[0.8080078959465027,
 0.6589590311050415,
 0.7827509045600891,
 0.8446911573410034,
 0.8253642916679382,
 0.8066179156303406,
 0.8760326504707336,
 0.8127508163452148,
 0.8029091358184814,
 0.773499071598053,
 0.8895395398139954,
 0.6910153031349182,
 0.8888853192329407,
 0.813069760799408,
 0.6996698975563049,
 0.8476549983024597,
 0.9170989394187927,
 0.8156989216804504,
 0.7129332423210144,
 0.692522406578064,
 0.8163334727287292,
 0.718087911605835,
 0.8134343028068542,
 0.8404116630554199,
 0.7938787937164307,
 0.8709847331047058,
 0.8363146781921387,
 0.6452261209487915,
 0.718925952911377,
 0.8686188459396362,
 0.8299380540847778,
 0.8682169318199158,
 0.8374934792518616,
 0.8109815716743469,
 0.8171431422233582,
 0.7192045450210571,
 0.6047594547271729,
 0.5446777939796448,
 0.7672617435455322,
 0.8212751746177673,
 0.6985032558441162,
 0.6581689119338989,
 0.7104586362838745,
 0.8273010849952698,
 0.6729130148887634,
 0.7186986804008484,
 0.8335407972335815,
 0.774374425411224

In [168]:
len(m_bayes)

100

In [171]:
np.sign(m_bayes)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Gathering all message iterations on functions:

In [178]:
def m_to_mhat(N, M, J_, m, beta, encoding):
    
    for mu in range(M):
        for j in range(N):
                     
            # Keep only L(mu) which a are different of j
            index_no_j = torch.nonzero(encoding[mu] != j).squeeze()
            L_no_j = encoding[mu][index_no_j]
        
            # Message update        
            m_hat[mu, j] = torch.tanh( beta* J_[mu])*torch.take(m[mu], L_no_j).prod(dim=0)
            
    return m_hat


#####################################################################


def mhat_to_m(N, M, m_hat, beta_prior, encoding):
    
    for j in range(N):
        for mu in range(M):
        
            M_set = torch.where(encoding == j)[0]
        
            M_set_no_mu = M_set[torch.nonzero(M_set != mu).squeeze()]
        
            m[mu, j] = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set_no_mu).sum() + beta_prior)
            
    return m


#####################################################################


def m_bayes(N, m_hat_final, beta_prior, encoding):
    
    m_bayes = []

    for j in range(N):
           
        M_set = torch.where(encoding == j)[0]
       
        m_j = torch.tanh(torch.take(np.arctanh(m_hat_final[:, j]), M_set).sum() + beta_prior)
    
        m_bayes.append(m_j.item())
        
    return m_bayes

Now we construct a function for the iterative decoding.

In [234]:
def BP_LDPC(N, M, J_, beta, beta_prior, encoding, message, num_it= 100, verbose= 1):
     
    m_hat = torch.rand(M, N)
    m = torch.empty(M, N)
    
    for j in range(num_it):
        
        print('--it = %d' % j)
              
        m = mhat_to_m(N, M, m_hat, beta_prior, encoding)
        m_hat = m_to_mhat(N, M, J_, m, beta, encoding)
        
        ## Monitoring performace
        if (j % verbose) == 0:
            m_ = m_bayes(N, m_hat, beta_prior, encoding)
            print('overlap= ', torch.dot(message, torch.Tensor(np.sign(m_))).item() / N)
        
    m_final = m_bayes(N, m_hat, beta_prior, encoding)
    
    return np.sign(m_final)

In [236]:
opt_dec_Bayes = BP_LDPC(N, M, J_, beta, beta_prior, encoding, message[0], num_it= 20, verbose= 1)

--it = 0
overlap=  0.64
--it = 1
overlap=  0.66
--it = 2
overlap=  0.7
--it = 3
overlap=  0.7
--it = 4
overlap=  0.7
--it = 5
overlap=  0.68
--it = 6
overlap=  0.68
--it = 7
overlap=  0.68
--it = 8
overlap=  0.68
--it = 9
overlap=  0.68
--it = 10
overlap=  0.68
--it = 11
overlap=  0.68
--it = 12
overlap=  0.68
--it = 13
overlap=  0.68
--it = 14
overlap=  0.68
--it = 15
overlap=  0.68
--it = 16
overlap=  0.68
--it = 17
overlap=  0.68
--it = 18
overlap=  0.68
--it = 19
overlap=  0.68


Ok. It still slow (we will worrie about this latter), but it appears to work.

Observe that we arbitrarily consider a certain `num_it`. Naturally, a important question remains: how do we monitor the convergence of BP?